In [1]:
from keras import layers
from keras import models

Using TensorFlow backend.


In [2]:
model = models.Sequential()

'''
    Данная сеть принимает на входе
    тензор с формой (высота_изображения,
    ширина_изображения, каналы).
    
    Изменение высоты и ширины сжимаются
    с ростом глубины сети. Количество каналов
    управляется первым арнументом,
    передаваемым в слои Conc2D (32 / 64).
'''

# сверточная состовляющая
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [3]:
'''
    Теперь необходимо передать после
    выходного тензора с формой (3, 3, 64)
    на вход полносвязной класифицирующей сети.
    
    Выходы (3, 3, 64) преобразуются в векторы
    (576,) перед передачей двум слоям Dense.
'''

# полносвязная сеть
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

In [4]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [5]:
# Сбор и подготовка данных к обучению и тестировке
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [6]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 38s 639us/step - loss: 0.1709 - accuracy: 0.9464
Epoch 2/5
60000/60000 [==============================] - 38s 637us/step - loss: 0.0461 - accuracy: 0.9855
Epoch 3/5
60000/60000 [==============================] - 43s 710us/step - loss: 0.0321 - accuracy: 0.9901
Epoch 4/5
60000/60000 [==============================] - 40s 663us/step - loss: 0.0233 - accuracy: 0.9926
Epoch 5/5
60000/60000 [==============================] - 40s 661us/step - loss: 0.0196 - accuracy: 0.9938


In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 2s 164us/step


0.9902999997138977

In [9]:
# Модель без max-polling
'''
    1. Данная модель не способствует изучению
    пространственной иерархии признаков.
    
    2. Заключительная карта признаков имеет 
    22 * 22 * 64 = 30976 признаков на образец.
'''
model_no_max_poll = models.Sequential()
model_no_max_poll.add(layers.Conv2D(32, (3, 3),
                      activation='relu',
                      input_shape=(28, 28, 1)))
model_no_max_poll.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_no_max_poll.add(layers.Conv2D(64, (3, 3), activation='relu'))

model_no_max_poll.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 22, 22, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________
